# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [33]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [34]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [35]:
def build_classifiers(combination):
    
    return combination[0].fit(data_set,labels), combination[1].fit(data_set,labels), combination[2].fit(data_set,labels), combination[3]


In [36]:
def build_stacked_classifier(classifiers):
    output = []
    stacked_classifier = classifiers[-1]
    classifiers = classifiers[:3]
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [37]:
list_of_function = [LinearRegression(), KNeighborsClassifier(), SVC(gamma='auto'), DecisionTreeClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis()]
from itertools import combinations
import operator
combinations = list(combinations(list_of_function, 4))
# print(combinations)
dict = {}

for combination in combinations:
    classifiers = build_classifiers(combination)
    predicted = build_stacked_classifier(classifiers)
    accuracy = accuracy_score(test_labels, predicted)
    name_list = [type(item).__name__ for item in combination]
    dict[' '.join(name_list)] = accuracy
#     print("give accuracy:", accuracy, "by models:", name_list)

    
maxv = max(dict.values())
new_dict = [k for k, v in dict.items() if v == maxv]
print("Max accuracy by this funcions, last one is used as stacked_classifier:")
print(new_dict)
print("If there was more than one function with max accuracy, they were listed as well")
print("With value:", maxv)

Max accuracy by this funcions, last one is used as stacked_classifier:
['LinearRegression KNeighborsClassifier SVC GaussianNB', 'LinearRegression KNeighborsClassifier DecisionTreeClassifier GaussianNB', 'LinearRegression SVC DecisionTreeClassifier GaussianNB', 'KNeighborsClassifier SVC DecisionTreeClassifier GaussianNB']
If there was more than one function with max accuracy, they were listed as well
With value: 1.0


C:\Users\Sebastian\scoop\apps\anaconda3\current\envs\datascience\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Sebastian\scoop\apps\anaconda3\current\envs\datascience\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Sebastian\scoop\apps\anaconda3\current\envs\datascience\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Sebastian\scoop\apps\anaconda3\current\envs\datascience\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Sebastian\scoop\apps\anaconda3\current\envs\datascience\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are colline

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [41]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

Fill the two functions below:

In [42]:
def set_new_weights(model):
    new_weights = []
    predicted = model.predict(test_set)
    I=np.array(calculate_accuracy_vector(predicted, test_labels))
    I = I + 1.0
    sumI = np.sum(I)
    for i in range(len(I)):
        new_weights.append(I[i]/sumI)
    return np.array(new_weights)


Train the classifier with the code below:

In [43]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

validate_x, validate_label = generate_data(1, dimension, labels)

Set the validation data set:

In [44]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [51]:
def get_prediction(x):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(x))
    output = np.array(output)
    predicted = []
    for i in range(len(x)):
        classified = output[:, i]
        counts = np.bincount(classified)
        predicted.append(np.argmax(counts))
    return predicted

Test it:

In [52]:
prediction = get_prediction(validate_x)[0]

print(prediction)

0
